In [1]:
 # importing necessary libraries
import pandas as pd
import psycopg2
# Database connection parameters
db_params = {
    "host": "localhost",     
    "database": "postgres",   
    "user": "admin",     
    "password": "123"  
}

In [2]:
#connecting to the postgre server with (try and except) to handel errors
try:
    connection = psycopg2.connect(**db_params)
    connection.autocommit = True
    cursor = connection.cursor()
    print("Connection to PostgreSQL successful")
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

# Creating the "worldcup" database
try:
    create_db_query = "CREATE DATABASE worldcup;"
    cursor.execute(create_db_query)
    print("Database 'worldcup' created successfully")
except (Exception, psycopg2.Error) as error:
    print("Error while creating 'worldcup' database:", error)
connection.close()    
# connecting again but using worldcup database instead of master    
db_params['database']='worldcup'
connection = psycopg2.connect(**db_params)
connection.autocommit = True
cursor = connection.cursor()
# creating the tables 
try:
    create_table_query = """
    CREATE TABLE worldcups (
    year int primary key,
    host_country varchar(40),
    winner varchar(40),
    runners_up varchar(40),
    third varchar(40),
    fourth varchar(40),
    goals_scored smallint,
    qualified_teams smallint,
    Matches_Played smallint,
    top_scorer varchar(50),
    best_player varchar(50),
    goals smallint,
    attendance int
    );
    CREATE TABLE matches (
        match_id SERIAL PRIMARY KEY,
        match_date date,
        stage VARCHAR(60),
        home_team varchar(40),
        away_team varchar(40),
        home_goals smallint,
        away_goals smallint,
        win_Conditions VARCHAR(80),
        year int REFERENCES worldcups(year)
    );
    """
    cursor.execute(create_table_query)
    print("Tables created successfully")
except (Exception, psycopg2.Error) as error:
    print("Error while creating 'matches and worldcups' table:", error)
# Closing the cursor and connection
cursor.close()
connection.close()

Connection to PostgreSQL successful
Database 'worldcup' created successfully
Tables created successfully


In [3]:
# loading dataset using pandas
WorldCups=pd.read_csv('WorldCups.csv', encoding='ISO-8859-1')
matches=pd.read_csv('world_cup_matches.csv', encoding='ISO-8859-1')

In [4]:
# inserting the data into worldcups table 
connection = psycopg2.connect(**db_params)
connection.autocommit = True
# Insert the data into the PostgreSQL table
try:
    with connection.cursor() as cursor:
        for _, row in WorldCups.iterrows():
            insert_query = """
              INSERT INTO worldcups (
                year, host_country, winner, runners_up, third, fourth, goals_scored,
                qualified_teams, matches_played, top_scorer,best_player, goals,attendance)
              VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                            """ 
            cursor.execute(insert_query, tuple(row))
    print('data inserted successfully ') 
except(Exception, psycopg2.Error) as error:
    print("Error while inserting date into 'worldcups' table:", error)    

data inserted successfully 


In [6]:
# inserting the data into worldcups table
matches=matches[['Date','Stage', 'Home Team', 'Away Team','Home Goals','Away Goals', 'Win Conditions','Year']]
try:
    with connection.cursor() as cursor:
        for _, row in matches.iterrows():
            insert_query = """
              INSERT INTO matches (
                match_date,stage, home_team, away_team, home_goals,
                away_goals, win_Conditions, year)
              VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                           """
            cursor.execute(insert_query, tuple(row))
    print('data inserted successfully ')            
except(Exception, psycopg2.Error) as error:
    print("Error while inserting date into 'matches' table:", error)

data inserted successfully 


In [10]:
#selecting first 10 row in table matches
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM matches LIMIT 10")
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,match_id,match_date,stage,home_team,away_team,home_goals,away_goals,win_conditions,year
0,1,1930-07-13,Group stage,France,Mexico,4,1,NaN,1930
1,2,1930-07-13,Group stage,United states,Belgium,3,0,NaN,1930
2,3,1930-07-14,Group stage,Yugoslavia,Brazil,2,1,NaN,1930
3,4,1930-07-14,Group stage,Romania,Peru,3,1,NaN,1930
4,5,1930-07-15,Group stage,Argentina,France,1,0,NaN,1930
5,6,1930-07-16,Group stage,Chile,Mexico,3,0,NaN,1930
6,7,1930-07-17,Group stage,Yugoslavia,Bolivia,4,0,NaN,1930
7,8,1930-07-17,Group stage,United states,Paraguay,3,0,NaN,1930
8,9,1930-07-18,Group stage,Uruguay,Peru,1,0,NaN,1930
9,10,1930-07-19,Group stage,Chile,France,1,0,NaN,1930


In [9]:
#selecting first 10 row in table worldcups
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM worldcups LIMIT 10")
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,host_country,winner,runners_up,third,fourth,goals_scored,qualified_teams,matches_played,top_scorer,best_player,goals,attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,Guillermo Stábile,José Nasazzi,8,590549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,Oldrich Nejedlý,Giuseppe Meazza,5,363000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,Leônidas,Leônidas,7,375700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,Ademir,Zizinho,8,1045246
4,1954,Switzerland,Germany,Hungary,Austria,Uruguay,140,16,26,Sándor Kocsis,Ferenc Puskas,11,768607
5,1958,Sweden,Brazil,Sweden,France,Germany,126,16,35,Just Fontaine,Didi,13,819810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,Garrincha,Garrincha,4,893172
7,1966,England,England,Germany,Portugal,Soviet Union,89,16,32,Eusébio,Bobby Charlton,9,1563135
8,1970,Mexico,Brazil,Italy,Germany,Uruguay,95,16,32,Gerd Müller,Pelé,10,1603975
9,1974,Germany,Germany,Netherlands,Poland,Brazil,97,16,38,Grzegorz Lato,Johan Cruyff,7,1865753


In [11]:
#Which Countries Hosted the World Cup and Won?
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT year, winner
        FROM worldcups 
        WHERE winner = host_country
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,winner
0,1930,Uruguay
1,1934,Italy
2,1966,England
3,1974,Germany
4,1978,Argentina
5,1998,France


In [12]:
#which world cup has highest goals scored?
with connection.cursor() as cursor:
    cursor.execute("""
        select year,host_country,goals_scored
        from worldcups
        order by goals_scored desc
        limit 1;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,host_country,goals_scored
0,2022,Qatar,172


In [13]:
#which country scored a highest goal outside home?
with connection.cursor() as cursor:
    cursor.execute("""
        select year,away_team as country ,away_goals as goals,
        home_team as against
        from matches
        order by away_goals desc
        limit 1;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,country,goals,against
0,1954,Austria,7,Switzerland


In [14]:
#which Final match has a highest-goals scored?
with connection.cursor() as cursor:
    cursor.execute("""
        select year,home_team,home_goals,away_goals,away_team
        from matches 
        where matches.stage='Final'
        order by home_goals+away_goals desc
        limit 1;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,home_team,home_goals,away_goals,away_team
0,1958,Sweden,2,5,Brazil


In [15]:
#Most Teams Won The World Cup. (top 5 Teams Won The World Cup)?
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT winner as country,count(winner) AS "world cups won",
        string_agg(DISTINCT year::text, ', ') AS years
        FROM worldcups
        GROUP BY winner
        ORDER BY count(winner) DESC
        LIMIT 5;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,country,world cups won,years
0,Brazil,5,"1958, 1962, 1970, 1994, 2002"
1,Germany,4,"1954, 1974, 1990, 2014"
2,Italy,4,"1934, 1938, 1982, 2006"
3,Argentina,3,"1978, 1986, 2022"
4,France,2,"1998, 2018"


In [16]:
#Most Teams achieve second place in The World Cup.
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT runners_up as country,count(runners_up) AS "runners_up",
        string_agg(DISTINCT year::text, ', ') AS years
        FROM worldcups
        GROUP BY runners_up
        ORDER BY count(runners_up) DESC
        LIMIT 5;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,country,runners_up,years
0,Germany,4,"1966, 1982, 1986, 2002"
1,Argentina,3,"1930, 1990, 2014"
2,Netherlands,3,"1974, 1978, 2010"
3,Czechoslovakia,2,"1934, 1962"
4,France,2,"2006, 2022"


In [17]:
#Most Teams achieve third place in The World Cup
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT third as country,count(third) AS "third place",
        string_agg(DISTINCT year::text, ', ') AS years
        FROM worldcups
        GROUP BY third
        ORDER BY count(third) DESC
        LIMIT 5;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,country,third place,years
0,Germany,4,"1934, 1970, 2006, 2010"
1,Brazil,2,"1938, 1978"
2,France,2,"1958, 1986"
3,Croatia,2,"1998, 2022"
4,Poland,2,"1974, 1982"


In [18]:
#which countries won the world cup in 2000s?
with connection.cursor() as cursor:
    cursor.execute("""
        select winner,year
        from worldcups
        where year>=2000
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,winner,year
0,Brazil,2002
1,Italy,2006
2,Spain,2010
3,Germany,2014
4,France,2018
5,Argentina,2022


In [19]:
#Average Goals Scored In each World Cups.
with connection.cursor() as cursor:
    cursor.execute("""
     select year,round(avg(home_goals+away_goals),1) as "average goals"
     from matches 
     group by year
     order by year 
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,average goals
0,1930,3.9
1,1934,4.1
2,1938,4.7
3,1950,4.0
4,1954,5.4
5,1958,3.6
6,1962,2.8
7,1966,2.8
8,1970,3.0
9,1974,2.6


In [20]:
#Which teams played most matches and how many goals did they score?
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT team,COUNT(team) AS match_count,
        SUM(total_goals) AS total_goals
        FROM (
              SELECT home_team AS team, home_goals AS total_goals
              FROM matches
              UNION ALL
              SELECT away_team AS team, away_goals AS total_goals
              FROM matches
              ) AS teams_goals
        GROUP BY team
        ORDER BY match_count DESC,total_goals desc
        LIMIT 5;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,team,match_count,total_goals
0,Brazil,114,237
1,Germany,112,232
2,Argentina,88,152
3,Italy,83,128
4,England,74,104


In [21]:
#MOST SCORED MATCH.
with connection.cursor() as cursor:
    cursor.execute("""
        select year,home_team , home_goals,away_goals,away_team
        from matches
        order by (home_goals+away_goals) desc
        limit 1;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,home_team,home_goals,away_goals,away_team
0,1954,Switzerland,5,7,Austria


In [22]:
#Top 5 Countries Scored Goals
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT team,SUM(total_goals) AS total_goals
        FROM (
              SELECT home_team AS team, home_goals AS total_goals
              FROM matches
              UNION ALL
              SELECT away_team AS team, away_goals AS total_goals
              FROM matches
              ) AS teams_goals
        GROUP BY team
        ORDER BY total_goals desc
        LIMIT 5;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,team,total_goals
0,Brazil,237
1,Germany,232
2,Argentina,152
3,France,136
4,Italy,128


In [23]:
#Number Of Appearance Of brazil For Each Cup and which year did it won.
with connection.cursor() as cursor:
    cursor.execute("""
        select matches.year,count(*) as "Num Of Appearance Of brazil",
        max(case when worldcups.winner='Brazil' then 'yes'
            else 'no' end) as "Won the world cup"
        from matches join worldcups
        on worldcups.year=matches.year
        where home_team = 'Brazil' or away_team='Brazil'
        group by matches.year
        order by matches.year;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,year,Num Of Appearance Of brazil,Won the world cup
0,1930,2,no
1,1934,1,no
2,1938,5,no
3,1950,6,no
4,1954,3,no
5,1958,6,yes
6,1962,6,yes
7,1966,3,no
8,1970,6,yes
9,1974,7,no


In [24]:
#Which team has the highest win percentage in all World Cup matches(list top 10)
with connection.cursor() as cursor:
    cursor.execute("""
     SELECT country,SUM(played) AS played,SUM(won) AS won,
     ROUND((SUM(won)*100)/SUM(played),1) as percentage
     FROM ( 
           SELECT home_team AS country, 1 AS played, 
           (CASE WHEN home_goals > away_goals THEN 1 ELSE 0 END) AS won
           FROM matches
           
           UNION ALL
           
           SELECT away_team AS country, 1 AS played, 
           (CASE WHEN home_goals < away_goals THEN 1 ELSE 0 END) AS won
           FROM matches
           ) AS winning
     GROUP BY country
     ORDER BY percentage DESC,won desc
     limit 10;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
df

,country,played,won,percentage
0,Brazil,114,76,66.0
1,Germany,112,68,60.0
2,Italy,83,45,54.0
3,Netherlands,55,30,54.0
4,Argentina,88,47,53.0
5,France,73,39,53.0
6,Turkey,10,5,50.0
7,Portugal,35,17,48.0
8,Spain,67,31,46.0
9,Hungary,32,15,46.0


In [25]:
#Which teams has the lowest win percentage in all World Cup
with connection.cursor() as cursor:
    cursor.execute("""
     SELECT country,SUM(played) AS played,SUM(won) AS won,
     ROUND((SUM(won)*100)/SUM(played),1) as percentage
     FROM ( 
           SELECT home_team AS country, 1 AS played, 
           (CASE WHEN home_goals > away_goals THEN 1 ELSE 0 END) AS won
           FROM matches
           
           UNION ALL
           
           SELECT away_team AS country, 1 AS played, 
           (CASE WHEN home_goals < away_goals THEN 1 ELSE 0 END) AS won
           FROM matches
           ) AS winning
     GROUP BY country
     ORDER BY percentage,played desc
     limit 10;
    """)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
    cursor.close()
    connection.close()
df

,country,played,won,percentage
0,Honduras,9,0,0.0
1,Egypt,7,0,0.0
2,Canada,6,0,0.0
3,New zealand,6,0,0.0
4,El salvador,6,0,0.0
5,Bolivia,6,0,0.0
6,Kuwait,3,0,0.0
7,China,3,0,0.0
8,Trinidad and tobago,3,0,0.0
9,Iceland,3,0,0.0
